![](https://media.giphy.com/media/dVuyBgq2z5gVBkFtDc/giphy.gif)

**Coronaviruses are a large family of viruses which may cause illness in animals or humans. In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19.COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019.**
* [Source](https://www.who.int/news-room/q-a-detail/q-a-coronaviruses)

# Coronavirus in the World

In [ ]:
import pandas as pd 
cases = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")
import plotly.offline as py
import plotly.express as px


py.init_notebook_mode(connected=True)

grp = cases.groupby(['ObservationDate', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered'].max()
grp = grp.reset_index()
grp['Date'] = pd.to_datetime(grp['ObservationDate'])
grp['Date'] = grp['Date'].dt.strftime('%m/%d/%Y')
grp['Active'] = grp['Confirmed'] - grp['Recovered'] - grp['Deaths']
grp['Country'] =  grp['Country/Region']

fig = px.choropleth(grp, locations="Country", locationmode='country names', 
                     color="Confirmed", hover_name="Country/Region",hover_data = [grp.Recovered,grp.Deaths,grp.Active],projection="mercator",
                     animation_frame="Date",width=1000, height=700,
                     color_continuous_scale='Reds',
                     range_color=[1000,50000],

                     title='World Map of Coronavirus')

fig.update(layout_coloraxis_showscale=True)
py.offline.iplot(fig)

# **Related Work**
 * For Analysis on Coronavirus(Canada), Click [here](https://www.kaggle.com/vanshjatana/coronavirus-in-canada)
* For Analysis and Prediction on Coronavirus(Italy), Click [here](https://www.kaggle.com/vanshjatana/analysis-and-prediction-on-coronavirus-italy?scriptVersionId=29892166)
*  For Analysis and Prediction on Coronavirus(Iran), Click [here](https://www.kaggle.com/vanshjatana/analysis-and-prediction-on-coronavirus-iran)
*  For Machine Learning on Cornovirus, Click [here](https://www.kaggle.com/vanshjatana/machine-learning-on-coronavirus)
*  For report on Coronavirus, Click [here](https://www.researchgate.net/publication/339738108_Analysis_On_Coronavirus)

# **SOUTH KOREA**

                                         **Current Scenario**
The accumulated confirmation count in South korea is increasing exponentially after 22nd of feb. Till the date 15th of feb 2020 it was negligible. There are many reason for infection but most cases have similar reasons that they have visited to wuhan. South Korea has raised its coronavirus alert to the "highest level" as confirmed case numbers keep rising.South Korea has seen the largest number of confirmed cases after China. More than 4,800 people have confirmed the infection of coronavirus.  As the data set says the western part of the country is more affected. Daegu and Gyeongsangbukdo region of South Korea are highly infected by the coronavirus

**Libraries**

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
from datetime import date, timedelta
from sklearn.cluster import KMeans
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


# **Symptoms of Coronavirus**
 * [Source](http://en.wikipedia.org/wiki/Coronavirus_disease_2019)

In [ ]:
symptoms={'symptom':['Fever',
        'Dry cough',
        'Fatigue',
        'Sputum production',
        'Shortness of breath',
        'Muscle pain',
        'Sore throat',
        'Headache',
        'Chills',
        'Nausea or vomiting',
        'Nasal congestion',
        'Diarrhoea',
        'Haemoptysis',
        'Conjunctival congestion'],'percentage':[87.9,67.7,38.1,33.4,18.6,14.8,13.9,13.6,11.4,5.0,4.8,3.7,0.9,0.8]}

symptoms=pd.DataFrame(data=symptoms,index=range(14))
symptoms

**Bar Chart**

In [ ]:
fig = px.bar(symptoms[['symptom', 'percentage']].sort_values('percentage', ascending=False), 
             y="percentage", x="symptom", color='symptom', 
             log_y=True, template='ggplot2', title='Symptom of  Coronavirus')
fig.show()

**Pie Chart**

In [ ]:
fig = px.pie(symptoms,
             values="percentage",
             names="symptom",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

**Tree Plot**

In [ ]:
fig = px.treemap(symptoms, path=['symptom'], values='percentage',
                  color='percentage', hover_data=['symptom'],
                  color_continuous_scale='Rainbow')
fig.show()

In [ ]:

from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in symptoms.symptom)
wordcloud = WordCloud(max_words=200,colormap='Set3', background_color="white").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
comp = pd.read_excel('/kaggle/input/covid19327/COVID-19-3.27-top30-500.xlsx')

In [ ]:
df_patient = pd.read_csv("../input/patient/patient.csv")
df_route = pd.read_csv("../input/route/route.csv")

In [ ]:
weather = pd.read_csv("../input/weather/Weather.csv")

# **Looking into patient data**

In [ ]:
df_patient.head()  

1.  **id** the ID of the patient (n-th confirmed patient)
2.  **sex** the sex of the patient
3.  **birth_year** the birth year of the patient
4.  **country** the country of the patient
5.  **region** the region of the patient
6.  **group** the collective infection
7.  **infection_reason** the reason of infection
8.  **infection_order** the order of infection
9.  **infected_by** the ID of who has infected the patient
10. **contact_number** the number of contacts with people
11. **confirmed_date** the date of confirmation
12. **released_date** the date of discharge
13. **deceased_date** the date of decease
14. **state** isolated / released / deceased

**Checking null value**

In [ ]:
df_patient.isna().sum()


In [ ]:
df_patient['birth_year'] = df_patient.birth_year.fillna(0.0).astype(int)
df_patient['birth_year'] = df_patient['birth_year'].map(lambda val: val if val > 0 else np.nan)

In [ ]:
df_patient.confirmed_date = pd.to_datetime(df_patient.confirmed_date)
daily_count = df_patient.groupby(df_patient.confirmed_date).id.count()
accumulated_count = daily_count.cumsum()




**Age**

In [ ]:
df_patient['age'] = 2020 - df_patient['birth_year'] 
df_patient['age'].head()

**Age Range**

In [ ]:
import math
def group_age(age):
    if age >= 0: # not NaN
        if age % 10 != 0:
            lower = int(math.floor(age / 10.0)) * 10
            upper = int(math.ceil(age / 10.0)) * 10 - 1
            return f"{lower}-{upper}"
        else:
            lower = int(age)
            upper = int(age + 9) 
            return f"{lower}-{upper}"
    return "Unknown"


df_patient["age_range"] = df_patient["age"].apply(group_age)
df_patient["age_range"].head()

In [ ]:
patient=df_patient

**Preprocessing**

In [ ]:
date_cols = ["confirmed_date", "released_date", "deceased_date"]
for col in date_cols:
    patient[col] = pd.to_datetime(patient[col])

In [ ]:
patient["time_to_release_since_confirmed"] = patient["released_date"] - patient["confirmed_date"]

patient["time_to_death_since_confirmed"] = patient["deceased_date"] - patient["confirmed_date"]
patient["duration_since_confirmed"] = patient[["time_to_release_since_confirmed", "time_to_death_since_confirmed"]].min(axis=1)
patient["duration_days"] = patient["duration_since_confirmed"].dt.days
age_ranges = sorted(set([ar for ar in patient["age_range"] if ar != "Unknown"]))
patient["state_by_gender"] = patient["state"] + "_" + patient["sex"]


# **Confirmed Count**

In [ ]:
accumulated_count.plot()
plt.title('Accumulated Confirmed Count');

# **Time series Analysis**

In [ ]:
time = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")

In [ ]:
time = time[time['Country/Region']=='South Korea']

In [ ]:
time = time.tail(14)

In [ ]:
time.head()

# **Confrim vs Recovery vs Death**

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Confirmed,label="Confirm")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Confrim',fontsize=30)
plt.show()


plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Recovered,label="Recovery")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Recovery',fontsize=30)
plt.show()

plt.figure(figsize=(23,10))

plt.bar(time.ObservationDate, time.Deaths,label="Death")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Death',fontsize=30)
plt.show()



In [ ]:
plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Confirmed,label="Confirm")
plt.bar(time.ObservationDate, time.Recovered,label="Recovery")
plt.bar(time.ObservationDate, time.Deaths,label="Death")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Confrim vs Recovery vs Death',fontsize=30)
plt.show()

f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="ObservationDate", y="Confirmed", data=time,
             color="black",label = "Confirm")
ax=sns.scatterplot(x="ObservationDate", y="Recovered", data=time,
             color="red",label = "Recovery")
ax=sns.scatterplot(x="ObservationDate", y="Deaths", data=time,
             color="blue",label = "Death")
plt.plot(time.ObservationDate,time.Confirmed,zorder=1,color="black")
plt.plot(time.ObservationDate,time.Recovered,zorder=1,color="red")
plt.plot(time.ObservationDate,time.Deaths,zorder=1,color="blue")

In [ ]:
!pip install chart_studio


In [ ]:
pip install bubbly


# Recovery Rate

In [ ]:
from bubbly.bubbly import bubbleplot 
from plotly.offline import iplot
import chart_studio.plotly as py

figure = bubbleplot(dataset=time, x_column='Confirmed', y_column='Recovered', 
    bubble_column='ObservationDate',size_column='Deaths', color_column='ObservationDate', 
    x_title="Confirm", y_title="Recovered", title='Confirm vs Recovered',
     scale_bubble=3, height=650)

iplot(figure, config={'scrollzoom': True})

# Death Rate

In [ ]:
from bubbly.bubbly import bubbleplot 
from plotly.offline import iplot
import chart_studio.plotly as py

figure = bubbleplot(dataset=time, x_column='Confirmed', y_column='Deaths', 
    bubble_column='ObservationDate',size_column='Deaths', color_column='ObservationDate', 
    x_title="Confirm", y_title="Deaths", title='Confirm vs Deaths',
     scale_bubble=3, height=650)

iplot(figure, config={'scrollzoom': True})

In [ ]:
import plotly.offline as py


# Stauts of person

In [ ]:
Total_confirmed = time['Confirmed'].sum()
Total_recovered = time['Recovered'].sum()
Total_death = time['Deaths'].sum()
data = [['Confirmed', Total_confirmed], ['Recovered', Total_recovered], ['Death', Total_death]] 
df = pd.DataFrame(data, columns = ['state', 'count']) 
fig = px.pie(df,
             values="count",
             names="state",
             title="State of Patient",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

# **Daily Growth**

In [ ]:
time['Confirmed_new'] = time['Confirmed']-time['Confirmed'].shift(1)
time['Recovered_new'] = time['Recovered']-time['Recovered'].shift(1)
time['Deaths_new'] = time['Deaths']-time['Deaths'].shift(1)


In [ ]:
plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Confirmed_new,label="Confirm")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Confrim',fontsize=30)
plt.show()


plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Recovered_new,label="Recovery")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Recovery',fontsize=30)
plt.show()

plt.figure(figsize=(23,10))

plt.bar(time.ObservationDate, time.Deaths_new,label="Death")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Death',fontsize=30)
plt.show()



In [ ]:
f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="ObservationDate", y="Confirmed_new", data=time,
             color="black",label = "Confirm")
ax=sns.scatterplot(x="ObservationDate", y="Recovered_new", data=time,
             color="red",label = "Recovery")
ax=sns.scatterplot(x="ObservationDate", y="Deaths_new", data=time,
             color="blue",label = "Death")
plt.plot(time.ObservationDate,time.Confirmed_new,zorder=1,color="black")
plt.plot(time.ObservationDate,time.Recovered_new,zorder=1,color="red")
plt.plot(time.ObservationDate,time.Deaths_new,zorder=1,color="blue")

In [ ]:
global_data = time

# **Growth Rate and Ratio**

In [ ]:
# This functions smooths data, thanks to Dan Pearson. We will use it to smooth the data for growth factor.
def smoother(inputdata,w,imax):
    data = 1.0*inputdata
    data = data.replace(np.nan,1)
    data = data.replace(np.inf,1)
    #print(data)
    smoothed = 1.0*data
    normalization = 1
    for i in range(-imax,imax+1):
        if i==0:
            continue
        smoothed += (w**abs(i))*data.shift(i,axis=0)
        normalization += w**abs(i)
    smoothed /= normalization
    return smoothed

def growth_factor(confirmed):
    confirmed_iminus1 = confirmed.shift(1, axis=0)
    confirmed_iminus2 = confirmed.shift(2, axis=0)
    return (confirmed-confirmed_iminus1)/(confirmed_iminus1-confirmed_iminus2)

def growth_ratio(confirmed):
    confirmed_iminus1 = confirmed.shift(1, axis=0)
    return (confirmed/confirmed_iminus1)

# This is a function which plots (for in input country) the active, confirmed, and recovered cases, deaths, and the growth factor.
def plot_country_active_confirmed_recovered(country):
    
    # Plots Active, Confirmed, and Recovered Cases. Also plots deaths.
    country_data = global_data[global_data['Country/Region']==country]
    table = country_data.drop(['SNo','Province/State', 'Last Update'], axis=1)
    table['ActiveCases'] = table['Confirmed'] - table['Recovered'] - table['Deaths']
    table2 = pd.pivot_table(table, values=['ActiveCases','Confirmed', 'Recovered','Deaths'], index=['ObservationDate'], aggfunc=np.sum)
    table3 = table2.drop(['Deaths'], axis=1)
   
    # Growth Factor
    w = 0.5
    table2['GrowthFactor'] = growth_factor(table2['Confirmed'])
    table2['GrowthFactor'] = smoother(table2['GrowthFactor'],w,5)

    # 2nd Derivative
    table2['2nd_Derivative'] = np.gradient(np.gradient(table2['Confirmed'])) #2nd derivative
    table2['2nd_Derivative'] = smoother(table2['2nd_Derivative'],w,7)


    #Plot confirmed[i]/confirmed[i-1], this is called the growth ratio
    table2['GrowthRatio'] = growth_ratio(table2['Confirmed'])
    table2['GrowthRatio'] = smoother(table2['GrowthRatio'],w,5)
    
    #Plot the growth rate, we will define this as k in the logistic function presented at the beginning of this notebook.
    table2['GrowthRate']=np.gradient(np.log(table2['Confirmed']))
    table2['GrowthRate'] = smoother(table2['GrowthRate'],0.5,3)
    
    # horizontal line at growth rate 1.0 for reference
    x_coordinates = [1, 100]
    y_coordinates = [1, 1]
    f, ax = plt.subplots(figsize=(15,5))
    table2['Deaths'].plot(title='Deaths')
    plt.show()
    f, ax = plt.subplots(figsize=(15,5))
    table2['GrowthFactor'].plot(title='Growth Factor')
    plt.plot(x_coordinates, y_coordinates) 
    plt.show()
    f, ax = plt.subplots(figsize=(15,5))
    table2['2nd_Derivative'].plot(title='2nd_Derivative')
    plt.show()
    f, ax = plt.subplots(figsize=(15,5))
    table2['GrowthRatio'].plot(title='Growth Ratio')
    plt.plot(x_coordinates, y_coordinates)
    plt.show()
    f, ax = plt.subplots(figsize=(15,5))
    table2['GrowthRate'].plot(title='Growth Rate')
    plt.show()

    return 

In [ ]:
plot_country_active_confirmed_recovered('South Korea')


# **Current State of Patient**

In [ ]:
infected_patient = patient.shape[0]
rp = patient.loc[patient["state"] == "released"].shape[0]
dp = patient.loc[patient["state"] == "deceased"].shape[0]
ip = patient.loc[patient["state"]== "isolated"].shape[0]
rp=rp/patient.shape[0]
dp=dp/patient.shape[0]
ip=ip/patient.shape[0]
print("The percentage of recovery is "+ str(rp*100) )
print("The percentage of deceased is "+ str(dp*100) )
print("The percentage of isolated is "+ str(ip*100) )


In [ ]:
states = pd.DataFrame(patient["state"].value_counts())
states["status"] = states.index
states.rename(columns={"state": "count"}, inplace=True)

fig = px.pie(states,
             values="count",
             names="status",
             title="Current state of patients",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="value+percent+label")
fig.show()

**People who released**

In [ ]:
released = df_patient[df_patient.state == 'released']
released.head()

**People who are in isolated state**

In [ ]:
isolated_state = df_patient[df_patient.state == 'isolated']
isolated_state.head()

**Patient who died**

In [ ]:
dead = df_patient[df_patient.state == 'deceased']
dead.head()

# **Age distribution of the released**

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the released")
sns.kdeplot(data=released['age'], shade=True)

In [ ]:
agr = released[released.age_range!="Unknown"]
fig = px.pie(agr,
             names="age_range",
             title="Age of released person",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

# **Age distribution of isolated person**

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the isolated")
sns.kdeplot(data=isolated_state['age'], shade=True)

In [ ]:
agi = isolated_state[isolated_state.age_range!="Unknown"]
fig = px.pie(agi,
             names="age_range",
             title="Age of isolated person",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

# **Age distribution of death**

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the deceased")
sns.kdeplot(data=dead['age'], shade=True)

In [ ]:
agd = dead[dead.age_range!="Unknown"]
fig = px.pie(agd,
             names="age_range",
             title="Age of Dead person",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

                                      ** Age distribution of deceased **
 Here, the graph shows the age distribution of the deceased by gender. We can clearly see older people are more likely to be infected, especially older people with underlying lung disease and having problems in their respiration. The age group of 60 to 70 yr are more infected than the rest of the population. As  Dr. Steven Gambert, professor of medicine and director of geriatrics at the University of Maryland School of Medicine says “ Older people have  higher risk of underlying health conditions, older people are already under physical stress, and their immune systems, even if not significantly compromised, simply do not have the same “ability to fight viruses and bacteria”

In [ ]:
male_dead = dead[dead.sex=='male']
female_dead = dead[dead.sex=='female']

# **Age distribution of death by gender**

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the deceased by gender")
sns.kdeplot(data=female_dead['age'], label="Women", shade=True)
sns.kdeplot(data=male_dead['age'],label="Male" ,shade=True)



In [ ]:
plt.figure(figsize=(10,8))
sns.set_style("darkgrid")
sns.distplot(a=male_dead['age'], label="Men", kde=False)
sns.distplot(a=female_dead['age'], label="Women", kde=False)
plt.title("Age distribution of the deceased by sex")
plt.legend()

                                    **Gender distribution of deceased **
The graph shows the age distribution of deceased gender where men are more likely to die from coronavirus  than women. New research  has found that men, particularly middle-aged and older men, are having a harder time fighting off the virus than women. Chinese researchers found that while the infection rate among men and women is the same, the death rate among men is 2.8% compared with 1.7% for women. As there are a few reasons men are more likely to die from the new coronavirus. Women have a heightened immune response. During Middle East respiratory syndrome coronavirus 32% of men died of the infection compared with 25.8% of women. 

**Comparison of released and deceased by age**

In [ ]:
sns.kdeplot(data=dead['age'],label='deceased', shade=True)
sns.kdeplot(data=released['age'],label='released', shade=True)
sns.kdeplot(data=isolated_state['age'],label='released', shade=True)

# **Death by gender**

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Sex')
dead.sex.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=dead.groupby(['sex']).size().values,names=dead.groupby(['sex']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

# **Isolation by gender**

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Sex')
isolated_state.sex.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=isolated_state.groupby(['sex']).size().values,names=isolated_state.groupby(['sex']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

# **Recovery by gender**

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Sex')
released.sex.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=released.groupby(['sex']).size().values,names=released.groupby(['sex']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

# **Reason for the infection**

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Infection reason')
df_patient.infection_reason.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=df_patient.groupby(['infection_reason']).size().values,names=df_patient.groupby(['infection_reason']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

In [ ]:
inf_rsn =  df_patient['infection_reason']
inf_rsn  = inf_rsn.dropna()

In [ ]:

from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in inf_rsn)
wordcloud = WordCloud(max_words=200,colormap='Set3', background_color="white").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

**There are many effective reasons for the coronavirus infection. There is no specific antiviral treatment recommended for COVID-19. People with COVID-19 should receive supportive care to help relieve symptoms. For severe cases, treatment should include care to support vital organ functions.People who think they may have been exposed to COVID-19 should contact their healthcare provider immediately.  According to the dataset the major reason for the infection is being in contact with patients. There are so many different reasons for the cause of infection which are visits to countries and regions like daegu, wuhan, thailand, china, vietnam, italy.**

In [ ]:
sns.set_style("whitegrid")
sns.FacetGrid(df_patient,  size = 20)\
.map(plt.scatter, 'age', 'infection_reason')\
.add_legend()
plt.title('Age vs Infection Reason',fontsize=30)
plt.xticks(fontsize=12)
plt.yticks(fontsize=15)
plt.show()

# **State of Patient**

In [ ]:
sns.set(rc={'figure.figsize':(5,5)})
sns.countplot(x=df_patient['state'].loc[
    (df_patient['infection_reason']=='contact with patient')
])

In [ ]:
fig = px.pie( values=df_patient.groupby(['state']).size().values,names=df_patient.groupby(['state']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

**State of male patient**

In [ ]:
sns.set(rc={'figure.figsize':(5,5)})
sns.countplot(x=df_patient['state'].loc[(df_patient['sex']=="male")])

**State of female patient**

In [ ]:
sns.set(rc={'figure.figsize':(5,5)})
sns.countplot(x=df_patient['state'].loc[(df_patient['sex']=="female")])

The first case of coronavirus in the country on January 20. At the moment, 31 patients were discharged from hospitals after making a full recovery from the virus and 22 deaths are reported. In total, over 105 thousand people have been tested until now. 

# **State of patient(male and female) vs age **

In [ ]:
age_gender_hue_order =["isolated_female", "released_female", "deceased_female",
                       "isolated_male", "released_male", "deceased_male"]
custom_palette = sns.color_palette("Reds")[3:6] + sns.color_palette("Blues")[2:5]

plt.figure(figsize=(12, 8))
sns.countplot(x = "age_range",
              hue="state_by_gender",
              order=age_ranges,
              hue_order=age_gender_hue_order,
              palette=custom_palette,
              data=patient)
plt.title("State by gender and age", fontsize=16)
plt.xlabel("Age range", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(loc="upper right")
plt.show()

# **Region by age and state**

In [ ]:
sns.set_style("whitegrid")
sns.FacetGrid(df_patient, hue = 'state', size = 10)\
.map(plt.scatter, 'age', 'region')\
.add_legend()
plt.title('Region by age and state')
plt.show()

# **Looking route data**

In [ ]:
df_route.head()

**Checking if is there any null value**

In [ ]:
df_route.isna().sum()

In [ ]:
clus=df_route.loc[:,['id','latitude','longitude']]


**Checking for number of cluster**

In [ ]:
K_clusters = range(1,8)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = df_route[['latitude']]
X_axis = df_route[['longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.show()



As in this graph, after 3 score go to constant value, so we will go with 5 clusters

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(clus[clus.columns[1:3]])
clus['cluster_label'] = kmeans.fit_predict(clus[clus.columns[1:3]])
centers = kmeans.cluster_centers_
labels = kmeans.predict(clus[clus.columns[1:3]])

**Graphical representation of clusters**

In [ ]:
clus.plot.scatter(x = 'latitude', y = 'longitude', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=100, alpha=0.5)

**We will verify our clusters by putting values in world map by making use of folium library**

# **Affected place in world map**

In [ ]:

 import folium
southkorea_map = folium.Map(location=[36.55,126.983333 ], zoom_start=7,tiles='Stamen Toner')

for lat, lon,city in zip(df_route['latitude'], df_route['longitude'],df_route['city']):
    folium.CircleMarker([lat, lon],
                        radius=5,
                        color='red',
                      popup =('City: ' + str(city) + '<br>'),
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(southkorea_map)
southkorea_map

In [ ]:
cmap1 = df_route
cmap1  = cmap1.groupby(['date', 'province','latitude','longitude'])['id'].max()


cmap1 = cmap1.reset_index()
cmap1.head()
cmap1['size'] = cmap1['id']*900
cmap1
fig = px.scatter_mapbox(cmap1, lat="latitude", lon="longitude",
                     color="id", size='size',
                     color_continuous_scale='burgyl',
                     animation_frame="date", 
                     title='Spread total cases over time')
fig.update(layout_coloraxis_showscale=True)
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3)
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()


On the world map it can clearly be seen that the western side of the country is more affected. The number of cases from Daegu and Gyeongbuk accounts for almost 88 percent of the total infections. This is closely linked to a religious group called Shincheonji Church of Jesus based in Daegu, where a collective infection has occurred and is spreading throughout the country.  About 60 percent of all infections in South Korea were related to Shincheonji. There have also been several deaths from a hospital linked to this religious group. The government of South Korea announced that it will test all over 200 thousand members of the group for the coronavirus. Over 70 countries are restricting entry from South korea. South Korea has the most cases after China.  South Korea reported the first case on 19th of feb.The cases increased multifold on 21 February, the majority of them being reported in the city of Daegu. The increase was due to community spread, traced to a church in Shincheonji, alarming the Korean health authorities. Singapore too is facing a similar surge in community spread of the novel coronavirus. Seoul city mayor Park Won-soon demanded to prosecute some of the key leaders of the Shincheonji Church for hiding the identities of the suspects during contact tracing investigation.The alert level was further raised to Orange (Level 3) on 28 January. Furthermore, 29 national designated hospitals were announced to isolate the confirmed and suspected cases

# **Patient in city**

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Number patients in city')
df_route.city.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=df_route.groupby(['city']).size().values,names=df_route.groupby(['city']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in df_route.city)
wordcloud = WordCloud(max_words=200,colormap='Set3', background_color="white").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

# **Patients in Provience/State**

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Number patients in province')
df_route.province.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=df_route.groupby(['province']).size().values,names=df_route.groupby(['province']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in df_route.province)
wordcloud = WordCloud(max_words=200,colormap='Set3', background_color="white").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

**73.5% cases of Coronavirus cases are of state Gyeongii-do and Seoul**

**Places where people get affected**

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Visit')
df_route.visit.value_counts().plot.bar();


In [ ]:
fig = px.pie( values=df_route.groupby(['visit']).size().values,names=df_route.groupby(['visit']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in df_route.visit)
wordcloud = WordCloud(max_words=200,colormap='Set3', background_color="white").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

# **Time from confirmation to release or death**

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x="state",
            y="duration_days",
            order=["released", "deceased"],
            data=patient)
plt.title("Time from confirmation to release or death", fontsize=16)
plt.xlabel("State", fontsize=16)
plt.ylabel("Days", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# **Time from confirmation to release or death by gender**

In [ ]:
order_duration_sex = ["female", "male"]
plt.figure(figsize=(12, 8))
sns.boxplot(x="sex",
            y="duration_days",
            order=order_duration_sex,
            hue="state",            
            hue_order=["released", "deceased"],
            data=patient)
plt.title("Time from confirmation to release or death by gender",
          fontsize=16)
plt.xlabel("Gender", fontsize=16)
plt.ylabel("Days", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# **Time from confirmation to release or death**

In [ ]:
order_duration_age = sorted(patient["age_range"].unique())[:-1]
plt.figure(figsize=(12, 8))
sns.boxplot(x="age_range",
            y="duration_days",
            order=order_duration_age,
            hue="state",
            hue_order=["released", "deceased"],
            data=patient)
plt.title("Time from confirmation to release or death", fontsize=16)
plt.xlabel("Age Range", fontsize=16)
plt.ylabel("Days", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

As the suspects of coronavirus are kept under observation. After the observation it has been found that the people who are released or who have great chances of recovery had sustained  10-20 days in hospitals. On the other hand the people who are deceased were found dead in 0-4 days.The deadly Coronavirus which has spread to more than 70 countries, is wreaking havoc globally, killing over 3,000 people and infecting almost 90,000 people since first detected in China in December last year. While Australia and the US reported their first deaths yesterday,

In [ ]:
data = daily_count.resample('D').first().fillna(0).cumsum()
data = data[20:]
x = np.arange(len(data)).reshape(-1, 1)
y = data.values

# **Regression Model**

In [ ]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=[32, 32, 10], max_iter=50000, alpha=0.0005, random_state=26)
_=model.fit(x, y)

In [ ]:
test = np.arange(len(data)+7).reshape(-1, 1)
pred = model.predict(test)
prediction = pred.round().astype(int)
week = [data.index[0] + timedelta(days=i) for i in range(len(prediction))]
dt_idx = pd.DatetimeIndex(week)
predicted_count = pd.Series(prediction, dt_idx)

**Graphical representatoin of current confirmed and predicted confirmed**

In [ ]:
accumulated_count.plot()
predicted_count.plot()
plt.title('Prediction of Accumulated Confirmed Count')
plt.legend(['current confirmd count', 'predicted confirmed count'])
plt.show()

# **Prophet**

In [ ]:
prophet= pd.DataFrame(data)
prophet
pr_data = prophet.reset_index()
pr_data.columns = ['ds','y']
pr_data.head()

**Prediction**

In [ ]:
m=Prophet()
m.fit(pr_data)
future=m.make_future_dataframe(periods=15)
forecast=m.predict(future)
forecast

**Graphical Representation of Prediction**

In [ ]:
import plotly.offline as py

figure = plot_plotly(m, forecast)
py.iplot(figure) 

figure = m.plot(forecast,xlabel='Date',ylabel='Confirmed Count')


In [ ]:
figure=m.plot_components(forecast)


**Autoregressive integrated moving average(Arima)**

In [ ]:
confirm_cs = pd.DataFrame(data).cumsum()
arima_data = confirm_cs.reset_index()
arima_data.columns = ['confirmed_date','count']
arima_data.head()

**Model**

In [ ]:
model = ARIMA(arima_data['count'].values, order=(1, 2, 1))
fit_model = model.fit(trend='c', full_output=True, disp=True)
fit_model.summary()


***Plotting the prediction***

In [ ]:
fit_model.plot_predict()
plt.title('Forecast vs Actual')
pd.DataFrame(fit_model.resid).plot()


**Forecasting future Values**

In [ ]:
forcast = fit_model.forecast(steps=5)
pred_y = forcast[0].tolist()
pd.DataFrame(pred_y)

# **LSTM**

In [ ]:
dataset = pd.DataFrame(data)
dataset.columns = ['Confirmed']
dataset.head()

In [ ]:
data = np.array(dataset).reshape(-1, 1)
train_data = dataset[:len(dataset)-5]
test_data = dataset[len(dataset)-5:]

**Modeling**

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)
n_input =5
n_features =1
                             
generator = TimeseriesGenerator(scaled_train_data,scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(units = 50, return_sequences = True, input_shape = (n_input, n_features)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units = 50, return_sequences = True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units = 50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units = 1))
lstm_model.compile(optimizer = 'adam', loss = 'mean_squared_error')
lstm_model.fit(generator, epochs = 21)

**Plottting Loss**

In [ ]:
losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize = (30,4))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(np.arange(0,100,1))
plt.plot(range(len(losses_lstm)), losses_lstm)

In [ ]:
lstm_predictions_scaled = []

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

**Prediction**

In [ ]:
prediction = pd.DataFrame(scaler.inverse_transform(lstm_predictions_scaled))
prediction.head()

# **Looking into time data**

In [ ]:
df_time = pd.read_csv("../input/convid19/time.csv")

In [ ]:
df_time.head()

**Shape of data**

In [ ]:
df_time.shape

In [ ]:
confirm_perc=(df_time['acc_confirmed'].sum()+ df_time['new_confirmed'].sum())/(df_time['acc_test'].sum() + df_time['new_test'].sum())*100
released_perc=(df_time['acc_released'].sum()+ df_time['new_released'].sum())/(df_time['acc_test'].sum() + df_time['new_test'].sum())*100
deceased_perc=(df_time['acc_deceased'].sum()+ df_time['new_deceased'].sum())/(df_time['acc_test'].sum() + df_time['new_test'].sum())*100

In [ ]:
print("The percentage of confirm  is "+ str(confirm_perc) )
print("The percentage of released is "+ str(released_perc) )
print("The percentage of deceased is "+ str(deceased_perc) )

In [ ]:
df_time = df_time.tail(15)

# **Test vs Confirmed**

In [ ]:
plt.figure(figsize=(100,30))
plt.bar(df_time.date, df_time.acc_test,label="Test")
plt.bar(df_time.date, df_time.acc_confirmed, label = "Confirmed")
plt.xlabel('Date')
plt.ylabel("Count")
plt.title('Test vs Confirmed',fontsize=100)
plt.legend(frameon=True, fontsize=12)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(100, 30))
ax=sns.scatterplot(x="date", y="acc_test", data=df_time,
             color="blue")
ax=sns.scatterplot(x="date", y="acc_confirmed", data=df_time,
             color="orange")


plt.plot(df_time.date,df_time.acc_test,zorder=1)
plt.plot(df_time.date,df_time.acc_confirmed,zorder=1,color="orange")
plt.title('Test vs Confirmed',fontsize=100)



# **Graphical representaion of new People Test and Confirmed**

In [ ]:
plt.figure(figsize=(100,30))
plt.bar(df_time.date, df_time.new_test,label="Test")
plt.bar(df_time.date, df_time.new_confirmed, label = "Confirmed")
plt.xlabel('Date')
plt.ylabel("Count")
plt.title('New Test vs New Confirmed',fontsize=100)

plt.legend(frameon=True, fontsize=12)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(100, 30))
ax=sns.scatterplot(x="date", y="new_test", data=df_time,
             color="blue")
ax=sns.scatterplot(x="date", y="new_confirmed", data=df_time,
             color="orange")


plt.plot(df_time.date,df_time.new_test,zorder=1)
plt.plot(df_time.date,df_time.new_confirmed,zorder=1,color="orange")
plt.title('Test vs Confirmed',fontsize=100)


As the condition is very critical in South Korea nowadays. The graph shown here represents the number of test cases over the number of confirmed cases date wise. We can clearly see here upto date 28th of jan 2020 condition was under control but after 28th of jan to 6th of feb there is a slight increase in graph. After 6th the graph has been increased linearly for the test cases but remains zero for deceased. After screening around 20,000 people  it was found that around 200+ people were infected .

# **Graphical representaion of   Confirmed vs State of Patient **

In [ ]:
plt.figure(figsize=(100,30))
plt.bar(df_time.date, df_time.acc_confirmed, label = "Confirmed")
plt.bar(df_time.date, df_time.acc_released,label="released")
plt.bar(df_time.date, df_time.acc_deceased,label="deceased")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(100, 30))
ax=sns.scatterplot(x="date", y="new_confirmed", data=df_time,
             color="red",label = "confirmed")
ax=sns.scatterplot(x="date", y="new_released", data=df_time,
             color="blue",label = "released")
ax=sns.scatterplot(x="date", y="new_deceased", data=df_time,
             color="orange",label = "deceased")
plt.plot(df_time.date,df_time.new_released,zorder=1,color="blue")
plt.plot(df_time.date,df_time.new_deceased,zorder=1,color="orange")
plt.plot(df_time.date,df_time.new_confirmed,zorder=1,color="red")


# **Graphical representaion of  new Confirmed vs State of Patient **

In [ ]:
plt.figure(figsize=(100,30))
plt.bar(df_time.date, df_time.new_confirmed, label = "Confirmed")
plt.bar(df_time.date, df_time.new_released,label="released")
plt.bar(df_time.date, df_time.new_deceased,label="deceased")
plt.xlabel('Date')
plt.ylabel("Count")
plt.xticks(fontsize=35)
plt.yticks(fontsize=30)

plt.legend(frameon=True, fontsize=12)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(100, 30))
ax=sns.scatterplot(x="date", y="new_confirmed", data=df_time,
             color="red",label = "confirmed")
ax=sns.scatterplot(x="date", y="new_released", data=df_time,
             color="blue",label = "released")
ax=sns.scatterplot(x="date", y="new_deceased", data=df_time,
             color="orange",label = "deceased")
plt.plot(df_time.date,df_time.new_released,zorder=1,color="blue")
plt.plot(df_time.date,df_time.new_deceased,zorder=1,color="orange")
plt.plot(df_time.date,df_time.new_confirmed,zorder=1,color="red")


**Looking into Trend file**

In [ ]:
trend = pd.read_csv("../input/trend1/trend.csv")

In [ ]:
trend.head()

# **No of time Coronavirus search?**

In [ ]:
f, ax = plt.subplots(figsize=(15,2))
h=pd.pivot_table(trend,columns='date',values=["coronavirus"])
sns.heatmap(h,cmap=['skyblue','red','green'],linewidths=0.05)

****Description of Trending Word****

In [ ]:
trend.describe().T

**Trend data of the keywords searched in NAVER which is one of the largest portal in South Korea**

**Plotting Graph of keywork search per 100 people recently**

In [ ]:
trend_last30 = trend.tail(30)
tl3 = trend_last30

In [ ]:
f, ax = plt.subplots(figsize=(50, 20))
sns.set_style("dark")
ax=sns.scatterplot(x="date", y="coronavirus", data=tl3,
             color="black",label = "coronavirus")
ax=sns.scatterplot(x="date", y="flu", data=tl3,
             color="red",label = "flu")
ax=sns.scatterplot(x="date", y="cold", data=tl3,
             color="blue",label = "cold")
ax=sns.scatterplot(x="date", y="pneumonia", data=tl3,
             color="orange",label = "pneumonia")
plt.plot(tl3.date,tl3.coronavirus,zorder=1,color="black")
plt.plot(tl3.date,tl3.cold,zorder=1,color="blue")
plt.plot(tl3.date,tl3.pneumonia,zorder=1,color="orange")
plt.plot(tl3.date,tl3.flu,zorder=1,color="red")

# **Comparision with other Country**

In [ ]:
comp_table = pd.DataFrame(comp.describe().T)


In [ ]:
comp_table


In [ ]:
comp = comp.loc[:,["Korea, South","US","Italy","France","China","Japan"]]


In [ ]:
comp.plot()


****Weather in South Korea****

In [ ]:
weather = weather.loc[:,["province","avg_relative_humidity"]]

In [ ]:
weather = weather.rename(columns={"avg_relative_humidity":"humidity"})

In [ ]:
weather.head()

In [ ]:
weather_min = weather.groupby("province")[["humidity"]].min().reset_index()
weather_avg = weather.groupby("province")[["humidity"]].mean().reset_index()
weather_max = weather.groupby("province")[["humidity"]].max().reset_index()


**Description of Minimum Humidity**

In [ ]:
weather_min.describe().T

**Description of Average Humidity**

In [ ]:
weather_avg.describe().T

**Description of Maximum Humidity**

In [ ]:
weather_max.describe().T

**Minimum Humidity**

In [ ]:
fig = px.bar(weather_min[['province', 'humidity']].sort_values('humidity', ascending=False), 
             y="humidity", x="province", color='province', 
             log_y=True, template='ggplot2', title='Minimum humidity ')
fig.show()

In [ ]:

fig = px.treemap(weather_min, path=['province'], values='humidity',
                  color='humidity', hover_data=['province'],
                  color_continuous_scale='Rainbow')
fig.show()

In [ ]:
fig = px.pie(weather_min,
             values="humidity",
             names="province",
             title="Minimum Humidity",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

**Average Humidity**

In [ ]:
fig = px.bar(weather_avg[['province', 'humidity']].sort_values('humidity', ascending=False), 
             y="humidity", x="province", color='province', 
             log_y=True, template='ggplot2', title='Minimum humidity ')
fig.show()

In [ ]:

fig = px.treemap(weather_avg, path=['province'], values='humidity',
                  color='humidity', hover_data=['province'],
                  color_continuous_scale='Rainbow')
fig.show()

In [ ]:
fig = px.pie(weather_avg,
             values="humidity",
             names="province",
             title="Average Humidity",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

**Maximum Humidity**

In [ ]:
fig = px.bar(weather_max[['province', 'humidity']].sort_values('humidity', ascending=False), 
             y="humidity", x="province", color='province', 
             log_y=True, template='ggplot2', title='Minimum humidity ')
fig.show()

In [ ]:

fig = px.treemap(weather_max, path=['province'], values='humidity',
                  color='humidity', hover_data=['province'],
                  color_continuous_scale='burgyl')
fig.show()

In [ ]:
fig = px.pie(weather_max,
             values="humidity",
             names="province",
             title="Maximum Humidity",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

**Humidity**

In [ ]:
f, ax = plt.subplots(figsize=(38,10))
plt.plot(weather_min.province,weather_min.humidity,zorder=1,color="black")
plt.plot(weather_avg.province,weather_avg.humidity,zorder=1,color="red")
plt.plot(weather_max.province,weather_max.humidity,zorder=1,color="blue")

**Prevention**  
![](http://defiancelibrary.org/images/CoronaVirusInfo.jpg)
   To avoid the critical situation people are suggested to do following things  
*     Avoid contact with people who are sick. 
*     Avoid touching your eyes, nose, and mouth. 
*     Stay home when you are sick. 
*     Cover your cough or sneeze with a tissue, then throw the tissue in the trash. 
*    Clean and disinfect frequently touched objects and surfaces using a regular household  
*    Wash your hands often with soap and water, especially after going to the bathroom; before eating; and after blowing     your nose, coughing, or sneezing. If soap and water are not readily available, use an alcohol-based hand sanitizer.